In [ ]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

# 💾 Paths
BASE_DIR = "C:\\Users\\MANJU\\Desktop\\FYP_Diabetes\\data"
TRAIN_DIR = os.path.join(BASE_DIR, "train")
VAL_DIR = os.path.join(BASE_DIR, "val")
TEST_DIR = os.path.join(BASE_DIR, "test")

# 📐 Parameters
IMG_HEIGHT, IMG_WIDTH = 224, 224
BATCH_SIZE = 64
EPOCHS = 30

# 🔁 Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.3,
    horizontal_flip=True,
    brightness_range=[0.7, 1.3],
    fill_mode='nearest'
)
val_test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

val_generator = val_test_datagen.flow_from_directory(
    VAL_DIR,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_generator = val_test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# ⚙️ Load base MobileNetV2
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))
base_model.trainable = False  # Freeze initial layers

# 🔧 Add custom head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.4)(x)
output = Dense(2, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

# 🔨 Compile model
model.compile(optimizer=Adam(learning_rate=0.0003),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# 📦 Callbacks
checkpoint = ModelCheckpoint("best_mobilenetv2_model.h5", monitor='val_accuracy', save_best_only=True, verbose=1)
early_stop = EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=4, min_lr=1e-6, verbose=1)

# 🚀 First Training Phase
model.fit(
    train_generator,
    epochs=15,
    validation_data=val_generator,
    callbacks=[checkpoint, early_stop, lr_scheduler]
)

# 🔓 Unfreeze last layers of base model
for layer in base_model.layers[-30:]:
    layer.trainable = True

# 🔁 Recompile after unfreezing
model.compile(optimizer=Adam(learning_rate=0.00005),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# 🚀 Fine-Tune Phase
model.fit(
    train_generator,
    epochs=15,
    validation_data=val_generator,
    callbacks=[checkpoint, early_stop, lr_scheduler]
)

# ✅ Final Evaluation
loss, accuracy = model.evaluate(test_generator)
print(f"\n✅ Final Accuracy: {accuracy * 100:.2f}%")
model.save("final_mobilenetv2_finetuned_model.h5")


Found 112 images belonging to 2 classes.
Found 24 images belonging to 2 classes.
Found 24 images belonging to 2 classes.
Epoch 1/15
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.4766 - loss: 1.0120  
Epoch 1: val_accuracy improved from -inf to 0.50000, saving model to best_mobilenetv2_model.h5


2/2 ━━━━━━━━━━━━━━━━━━━━ 16s 5s/step - accuracy: 0.4844 - loss: 1.0066 - val_accuracy: 0.5000 - val_loss: 0.8131 - learning_rate: 3.0000e-04
Epoch 2/15
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.6280 - loss: 0.7636
Epoch 2: val_accuracy improved from 0.50000 to 0.66667, saving model to best_mobilenetv2_model.h5


2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 3s/step - accuracy: 0.6359 - loss: 0.7463 - val_accuracy: 0.6667 - val_loss: 0.6923 - learning_rate: 3.0000e-04
Epoch 3/15
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.6786 - loss: 0.6498
Epoch 3: val_accuracy improved from 0.66667 to 0.70833, saving model to best_mobilenetv2_model.h5


2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 3s/step - accuracy: 0.6756 - loss: 0.6515 - val_accuracy: 0.7083 - val_loss: 0.6275 - learning_rate: 3.0000e-04
Epoch 4/15
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.6886 - loss: 0.5962
Epoch 4: val_accuracy improved from 0.70833 to 0.79167, saving model to best_mobilenetv2_model.h5


2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 3s/step - accuracy: 0.6942 - loss: 0.5887 - val_accuracy: 0.7917 - val_loss: 0.5833 - learning_rate: 3.0000e-04
Epoch 5/15
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7381 - loss: 0.5754
Epoch 5: val_accuracy did not improve from 0.79167
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 3s/step - accuracy: 0.7480 - loss: 0.5717 - val_accuracy: 0.7917 - val_loss: 0.5633 - learning_rate: 3.0000e-04
Epoch 6/15
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.8516 - loss: 0.4198
Epoch 6: val_accuracy did not improve from 0.79167
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 2s/step - accuracy: 0.8385 - loss: 0.4332 - val_accuracy: 0.7917 - val_loss: 0.5639 - learning_rate: 3.0000e-04
Epoch 7/15
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7932 - loss: 0.6311
Epoch 7: val_accuracy did not improve from 0.79167
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 3s/step - accuracy: 0.7937 - loss: 0.6132 - val_accuracy: 0.7917 - val_loss: 0.5549 - learning_rate: 3.0000e-04
Epoch 8/15
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/st


✅ Final Accuracy: 70.83%
